Biostat/Biomath M257 Homework 3

Due May 3 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw3`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [31c24e10] Distributions v0.25.108
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In [3]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

**Solution**

Since $\boldsymbol{\gamma}$ is independent of $\boldsymbol{\epsilon}_i$, we have $\mathbf{y}_i \sim N(\mathbf{X}_i \boldsymbol{\beta}, \mathbf{\Omega}_i)$, where $\mathbf{\Omega}_i = \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i' + \sigma^2 \mathbf{I}_{n_i}$. Hence, the likelihood $L(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$ is written as
$$
    L(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = 
    (2\pi)^{-n_i/2}\vert\mathbf{\Omega}_i\vert^{-1/2}
    \exp\left[-\frac {(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})} 2\right],
$$
and thus the log-likelihood $\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$ is given by
$$
\begin{align}
    \ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) 
    &=
    - \frac{n_i} 2 \ln(2\pi)
    - \frac 1 2 \ln\vert\mathbf{\Omega}_i\vert
    - \frac 1 2 (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}).
\end{align}
$$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [4]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
    y :: Vector{T}, 
    X :: Matrix{T}, 
    Z :: Matrix{T}
    ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_q, xtx, ztx, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.

**Solution**

We need to consider efficient computation of $\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

Using the determinant property used in hw1 Q5.4, we have
$$
\begin{align*}
\vert\mathbf{\Omega}_i\vert
&= \vert\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i\boldsymbol{\Sigma}\mathbf{Z}_i'\vert\\
&= \sigma^{2n_i}\vert \mathbf{I}_{n_i} + \sigma^{-2}\mathbf{Z}_i\mathbf{L}\mathbf{L}'\mathbf{Z}_i'\vert\\
&= \sigma^{2n_i}\vert \mathbf{I}_{q} + \sigma^{-2}\mathbf{L}'\mathbf{Z}_i'\mathbf{Z}_i\mathbf{L}\vert
\end{align*}
$$
Hence, taking the log gives
$$
\begin{align*}
\ln\vert\mathbf{\Omega}_i\vert = n_i\ln\sigma^2 + \ln\vert \mathbf{I}_{q} + \sigma^{-2}\mathbf{L}'\mathbf{Z}_i'\mathbf{Z}_i\mathbf{L}\vert
\end{align*}
$$
Using the Binomial inversion formula shown in hw1 Q5.3, we have
$$
\begin{align*}
\mathbf{\Omega}_i^{-1}
&= (\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i\boldsymbol{\Sigma}\mathbf{Z}_i')^{-1}\\
&= (\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i\mathbf{L}\mathbf{L}'\mathbf{Z}_i')^{-1}\\
&= \sigma^{-2} \mathbf I_{q} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{I}_q + \sigma^{-2}\mathbf{L}'\mathbf{Z}_i'\mathbf{Z}_i\mathbf{L})^{-1}\mathbf{L}'\mathbf{Z}_i'
\end{align*}
$$
Then, the third term without the coefficient can be expanded as follows:
$$
\begin{align*}
(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})
= \sigma^{-2}\|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}\|_2^2
- \sigma^{-4}(\mathbf{y}_i
- \mathbf{X}_i \boldsymbol{\beta})'\mathbf{Z}_i\mathbf{L}
(\mathbf{I}_q + \sigma^{-2}\mathbf{L}'\mathbf{Z}_i'\mathbf{Z}_i\mathbf{L})^{-1}
\mathbf{L}'\mathbf{Z}_i'(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}),
\end{align*}
$$
where $\mathbf{L}'\mathbf{Z}_i'(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})\in \mathbb R^q$.


Finally, by another Cholesky decomposition $\mathbf{I}_q + \sigma^{-2}\mathbf{L}'\mathbf{Z}_i'\mathbf{Z}_i\mathbf{L} = \mathbf{M} \mathbf{M}'$, the log-likelihood $(1)$ can be changed to
$$
\begin{align*}
    \ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) 
    &= - \frac{n_i} 2 \ln(2\pi)
    - \frac 1 2\left(n_i \ln\sigma^2 + \ln\vert\mathbf{M}\mathbf{M}'\vert\right)\\
    &\quad - \frac 1 2 \left[\frac{\|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}\|_2^2}{\sigma^2}
    - \frac{(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{Z}_i\mathbf{L}
    (\mathbf{M}\mathbf{M}')^{-1}
    \mathbf{L}'\mathbf{Z}_i'(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})}{\sigma^4}\right]\\
    &= - \frac{n_i} 2 \ln(2\pi\sigma^2)
    - \ln\vert\mathbf{M}\vert\\
    &\quad - \frac 1 2\frac{\|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}\|_2^2}{\sigma^2}
    + \frac 1 2 \frac{\|\mathbf{M}^{-1}
    (\mathbf{L}'\mathbf{Z}_i'\mathbf{y}_i - \mathbf{L}'\mathbf{Z}_i'\mathbf{X}_i \boldsymbol{\beta})\|_2^2}{\sigma^4}.
\end{align*}
$$
In addition, the following expansion of the numarator in the 4th term reduces more time and memory:
$$
\|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}\|_2^2
= \|\mathbf{y}_i\|_2^2 + \boldsymbol{\beta}'(\mathbf{X}_i'\mathbf{X}_i\boldsymbol{\beta} - 2\mathbf{X}_i'\mathbf{y}_i),
$$
where $\mathbf{X}_i'\mathbf{X}_i\boldsymbol{\beta} - 2\mathbf{X}_i'\mathbf{y}_i\in\mathbb R^p$.

In [15]:
function logl!(
    obs :: LmmObs{T},
    β   :: Vector{T},
    L   :: Matrix{T}, # lower triangular Cholesky factor
    σ²  :: T
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    
    # compute and return the log-likelihood
    # LogLik = p
    LogLik = n * log(2π * σ²) + 2 * logdet(LowerTriangular(L)) + dot(obs.y, obs.y) / σ²

    obs.storage_qq .= LowerTriangular(L) \ I # L⁻¹
    mul!(obs.storage_qq, transpose(obs.storage_qq), obs.storage_qq)
    obs.storage_qq .+= obs.ztz / σ²

    # L = Matrix(cholesky(obs.storage_qq).L)
    # obs.storage_qq .= Symmetric(obs.storage_qq)
    LAPACK.potrf!('L', obs.storage_qq) 
    # LAPACK.potri!('L', obs.storage_qq) # L⁻¹ = LowerTriangular(L) \ I
    # L⁻¹ = LowerTriangular(L) \ I
    
    LogLik += 2 * logdet(LowerTriangular(obs.storage_qq)) # sum(log.(diag(L))) is slower
    # LogLik -= 2 * logdet(LowerTriangular(obs.storage_qq))

    mul!(obs.storage_p, transpose(obs.X), obs.y) # X'y
    # LogLik -= 2dot(β, obs.storage_p) / σ²
    # mul!(obs.storage_p, obs.xtx, β) # X'Xβ
    # LogLik += dot(β, obs.storage_p) / σ²
    BLAS.symv!('U', 1.0, obs.xtx, β, -2.0, obs.storage_p) # X'Xβ - 2X'y
    LogLik += dot(β, obs.storage_p) / σ²

    mul!(obs.storage_q, transpose(obs.Z), obs.y)
    obs.storage_q .-= obs.ztx * β  # Z'y - Z'Xβ
    mul!(obs.storage_q, LowerTriangular(obs.storage_qq) \ I, obs.storage_q)
    # mul!(obs.storage_q, LowerTriangular(obs.storage_qq), obs.storage_q)
    
    LogLik -= dot(obs.storage_q, obs.storage_q) / σ²^2

    LogLik *= -0.5
    # LogLik = p # same place
    # LogLik = -0.5n * log(2π * σ²) - logdet(LowerTriangular(L)) - 
    #     logdet(LowerTriangular(obs.storage_qq)) - 
    #     0.5dot(obs.y, obs.y) / σ² - 
    #     0.5dot(β, obs.storage_p) / σ² + 
    #     0.5dot(obs.storage_q, obs.storage_q) / σ²^2

    return LogLik 
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return zero(T)
end

logl! (generic function with 1 method)

In [ ]:
# function logl!(
#     obs :: LmmObs{T},
#     β   :: Vector{T},
#     L   :: Matrix{T}, # lower triangular Cholesky factor
#     σ²  :: T
#     ) where T <: AbstractFloat
#     n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    
#     # compute and return the log-likelihood
#     # LogLik = p
#     LogLik = n * log(2π * σ²) + 2 * logdet(LowerTriangular(L)) + dot(obs.y, obs.y) / σ²

#     obs.storage_qq .= LowerTriangular(L) \ I # L⁻¹
#     mul!(obs.storage_qq, transpose(obs.storage_qq), obs.storage_qq)
#     obs.storage_qq .+= obs.ztz / σ²

#     # L = Matrix(cholesky(obs.storage_qq).L)
#     # obs.storage_qq .= Symmetric(obs.storage_qq)
#     LAPACK.potrf!('L', obs.storage_qq) 
#     # LAPACK.potri!('L', obs.storage_qq) # L⁻¹ = LowerTriangular(L) \ I
#     # L⁻¹ = LowerTriangular(L) \ I
    
#     LogLik += 2 * logdet(LowerTriangular(obs.storage_qq)) # sum(log.(diag(L))) is slower
#     # LogLik -= 2 * logdet(LowerTriangular(obs.storage_qq))

#     mul!(obs.storage_p, transpose(obs.X), obs.y) # X'y
#     # LogLik -= 2dot(β, obs.storage_p) / σ²
#     # mul!(obs.storage_p, obs.xtx, β) # X'Xβ
#     # LogLik += dot(β, obs.storage_p) / σ²
#     BLAS.symv!('U', 1.0, obs.xtx, β, -2.0, obs.storage_p) # X'Xβ - 2X'y
#     LogLik += dot(β, obs.storage_p) / σ²

#     mul!(obs.storage_q, transpose(obs.Z), obs.y)
#     obs.storage_q .-= obs.ztx * β  # Z'y - Z'Xβ
#     mul!(obs.storage_q, LowerTriangular(obs.storage_qq) \ I, obs.storage_q)
#     # mul!(obs.storage_q, LowerTriangular(obs.storage_qq), obs.storage_q)
    
#     LogLik -= dot(obs.storage_q, obs.storage_q) / σ²^2

#     LogLik *= -0.5
#     # LogLik = p # same place
#     # LogLik = -0.5n * log(2π * σ²) - logdet(LowerTriangular(L)) - 
#     #     logdet(LowerTriangular(obs.storage_qq)) - 
#     #     0.5dot(obs.y, obs.y) / σ² - 
#     #     0.5dot(β, obs.storage_p) / σ² + 
#     #     0.5dot(obs.storage_q, obs.storage_q) / σ²^2

#     return LogLik 
#     sleep(1e-3) # wait 1 ms as if your code takes 1ms
#     return zero(T)
# end

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [6]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.694969966642933, 1.7723256696372405, 1.1065838446466518, 3.729166811829607, 4.288899999400642, 2.8241842645202406  …  4.058027151891634, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.5193020855673436, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [5.0e-324, 1.0e-323, 2.2138350294e-314, 2.213835061e-314, 0.0], [2.201136894e-314, 2.201136902e-314, 0.0], [2000.0 -16.870943820386742 … -

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [7]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.1793358058317

Check that your answer matches that from Distributions.jl

In [16]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058317

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [52]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [18]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 2843 samples with 1 evaluation.
 Range (min … max):  979.583 μs …   7.013 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.649 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.737 ms ± 443.177 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

               ▃█▅▂▃    ▃▃▂▁                                     
  ▂▃▃▃▂▃▅▇▇▅▅▆███████▆▅▆█████▆▆▅▅▃▃▃▃▃▃▂▂▃▃▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▂▁▁▁▁ ▃
  980 μs           Histogram: frequency by time          3.2 ms <

 Memory estimate: 31.77 KiB, allocs estimate: 3.

In [17]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  10.125 μs … 30.573 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.938 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.912 μs ±  1.621 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▆▃ ▁▂▁             ▂▇█▆▂                            ▃▆▆▅▂ ▂
  ████▇████▇▄▃▆▃▃▁▁▁▁▄▅█████▇▇▇█▇█▆▁▃▃▃▄▃▃▄▁▄▁▃▃▄▁▁▁▁▁▁▇█████ █
  10.1 μs      Histogram: log(frequency) by time      14.8 μs <

 Memory estimate: 464 bytes, allocs estimate: 4.

In [58]:
L = Matrix(cholesky(Σ).L)
@benchmark transpose(inv(L)) * inv(L)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.475 μs …  42.850 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.739 μs ± 558.685 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▁    ▅█                                                    
  ▂██▃▂▃███▅▃▃▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.48 μs         Histogram: frequency by time        3.22 μs <

 Memory estimate: 400 bytes, allocs estimate: 4.

In [22]:
@benchmark inv(transpose(L)) * inv(L)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.775 μs …   9.267 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.875 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.917 μs ± 193.595 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▁▃█▄▂▂▃                                                
  ▂▂▃▄▇▇███████▆▅▄▄▃▃▃▃▄▄▄▄▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▂▁▁▂▂ ▃
  1.78 μs         Histogram: frequency by time        2.32 μs <

 Memory estimate: 416 bytes, allocs estimate: 5.

In [26]:
L⁻¹ = inv(L)
@benchmark transpose(L⁻¹) * L⁻¹

BenchmarkTools.Trial: 10000 samples with 883 evaluations.
 Range (min … max):   89.514 ns …   3.328 μs  ┊ GC (min … max): 0.00% … 96.18%
 Time  (median):      96.971 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   104.303 ns ± 123.430 ns  ┊ GC (mean ± σ):  4.84% ±  3.96%

               ▂▁▁ ▂█▇▃▂                                         
  ▁▁▁▁▁▁▁▁▁▁▂▃▅██████████▆▃▂▂▂▂▂▁▂▂▂▃▃▄▅▅▆▅▆▆▄▄▄▃▄▃▃▃▃▂▂▂▁▁▁▁▁▁ ▃
  89.5 ns          Histogram: frequency by time          111 ns <

 Memory estimate: 144 bytes, allocs estimate: 2.

In [141]:
@benchmark inv(LowerTriangular(L))

BenchmarkTools.Trial: 10000 samples with 106 evaluations.
 Range (min … max):  766.509 ns …  28.608 μs  ┊ GC (min … max): 0.00% … 96.16%
 Time  (median):     887.179 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   948.061 ns ± 353.281 ns  ┊ GC (mean ± σ):  0.29% ±  0.96%

  ▂▇       █▆                                                    
  ██▆▃▂▂▂▂▆██▇▃▂▂▂▂▂▂▄▅▃▃▂▂▂▂▂▂▂▂▂▁▃▅▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▂▄▅▃▂▂▂▂ ▃
  767 ns           Histogram: frequency by time         1.51 μs <

 Memory estimate: 160 bytes, allocs estimate: 3.

In [142]:
@benchmark L / I

BenchmarkTools.Trial: 10000 samples with 981 evaluations.
 Range (min … max):  71.738 ns …   4.038 μs  ┊ GC (min … max): 0.00% … 96.53%
 Time  (median):     92.380 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   99.223 ns ± 143.727 ns  ┊ GC (mean ± σ):  5.72% ±  3.86%

          ▂▂▆█▅▃▂   ▃▅▆▅▄▄▅▇▅▂▂▁   ▂▃▅▅▃▂▁     ▁▄▂          ▁▁ ▂
  ▆▁▁▁▁▃▅██████████▇█████████████▇▇██████████▆▇███▇▆▆▆▆▅▅▃▆███ █
  71.7 ns       Histogram: log(frequency) by time       130 ns <

 Memory estimate: 128 bytes, allocs estimate: 1.

In [152]:
@benchmark norm($y)^2

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.018 μs …  5.768 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.046 μs ± 51.855 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▄   ▇   █   ▇   ▄    ▁                             ▂
  ▃▁▁▁▅▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁▇█▁▁▁█▁▁▁▇▁▁▁▇▁▁▁▆▁▁▁▅▁▁▁▅▁▁▁▃ █
  4.02 μs      Histogram: log(frequency) by time      4.1 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [157]:
@benchmark dot($y, $y)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.425 μs …  2.508 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.438 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.439 μs ± 16.025 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▅      █      ▄▅      ▃      ▁               ▁
  ▃▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁██▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▇ █
  1.42 μs      Histogram: log(frequency) by time     1.46 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.425 μs …  2.867 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.438 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.439 μs ± 25.892 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▅       █        ▆       ▂                 ▁
  ▃▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▄ █
  1.42 μs      Histogram: log(frequency) by time     1.45 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [1]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

UndefVarError: UndefVarError: `median` not defined

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [357]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.546875

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.